##### 问题:(这一节问题较多，尚未明确)
我们的程序中创建了环状的数据结构（例如树、图、观察者模式等），但是在内存管理
上却遇到了麻烦。

##### 解决方案:
环状数据结构的一个简单例子就是树了，这里父节点指向它的孩子，而孩子节点又会
指回它们的父节点。对于像这样的代码，我们应该考虑让其中一条连接使用 weakref
库中提供的弱引用机制。示例如下：

In [1]:
import weakref
class Node:
    def __init__(self, value):
        self.value = value
        self._parent = None
        self.children = []

    def __repr__(self):
        return 'Node({!r:})'.format(self.value)

    # property that manages the parent as a weak-reference
    @property
    def parent(self):
        return self._parent if self._parent is None else self._parent()

    @parent.setter
    def parent(self, node):
        self._parent = weakref.ref(node)

    def add_child(self, child):
        self.children.append(child)
        child.parent = self 



这种实现可以让父节点安静地被回收。示例如下:

In [2]:
root = Node('parent')
c1 = Node('child')
root.add_child(c1)
print(c1.parent)
del root
print(c1.parent)

Node('parent')
None


环状数据结构是 Python 中一个多少需要一些技巧才能处理好的方面，需要仔细学习。
因为普通的垃圾收集规则并不适用于环状数据结构。例如，考虑下面的代码：

In [3]:
# Class just to illustrate when deletion occurs
class Data:
    def __del__(self):
        print('Data.__del__')
# Node class involving a cycle
class Node:
    def __init__(self):
        self.data = Data()
        self.parent = None
        self.children = []
    def add_child(self, child):
        self.children.append(child)
        child.parent = self

现在，试用上面的代码，做些试验来看看有关垃圾收集中的一些微妙问题：

In [4]:
a = Data()
del a                    # Immediately deleted#

Data.__del__


In [5]:
a = Node()
del a                   # Immediately deleted#

Data.__del__


In [6]:
a = Node()
a.add_child(Node())
print(a)
del a   # Not deleted (no message)
#但是如果打印a会被告知a未定义,我的理解是:a是Node类型,因为删除了部分(Data)但未完全删除a,导致a不完整,无法访问。
#又因为py在运行结束时会自动回收内存,所以可能会在py终端运行时输出两个Data.__del__
print('over')


over


可以看到，除了最后那种涉及成环的情况，其他的对象都可以立刻得到删除。原因在
于 Python 的垃圾收集器是基于简单的引用计数规则来实现的。当对象的引用计数为 0
时就会被立刻删除掉。而对于环状数据结构来说这绝不可能发生。因为在最后那种情
况中，由于父节点和子节点互相引用对方，引用计数不会为 0。

要处理环状数据结构，还有一个单独的垃圾收集器会定期运行。但是，一般来说我们不
知道它会在何时运行。因此，没法知道环状数据结构具体会在何时被回收。如果有必要
的话，可以强制运行垃圾收集器，但这么做相比于全自动的垃圾收集会有一些笨拙。

In [7]:
# Class just to illustrate when deletion occurs
class Data:
    def __del__(self):
        print('Data.__del__')
# Node class involving a cycle
class Node:
    def __init__(self):
        self.data = Data()
        self.parent = None
        self.children = []
    def add_child(self, child):
        self.children.append(child)
        child.parent = self

import gc
b = Node()
b.add_child(Node())
gc.collect()            # Force collection。注意这里只有jupyter第一次运行是可以成功,第二次运行就会报错,但毋庸置疑的是强制回收有效

Data.__del__
Data.__del__


177

如果环中的对象实现了自己的__del__方法的话，则情况会更糟。例如，假设有下面这
样的代码：

In [8]:
# Class just to illustrate when deletion occurs
class Data:
    def __del__(self):
        print('Data.__del__')
# Node class involving a cycle
class Node:
    def __init__(self):
        self.data = Data()
        self.parent = None
        self.children = []

    # NEVER DEFINE LIKE THIS.
    # Only here to illustrate pathological behavior
    def __del__(self):
        del self.data
        del parent
        del children
        print('Node.__del__')
    def add_child(self, child):
        self.children.append(child)
        child.parent = self 




在这种情况下，数据结构对象永远不会被垃圾收集，我们的程序会因此而出现内存泄
露！如果动手尝试一下，会发现 Data.__del__消息完全没有被打印出来——即使是强制
执行垃圾收集也不会：

In [9]:
c = Node()
c.add_child(Node())
import gc
gc.collect()

0

弱引用通过消除循环引用来解决这个问题。本质上说，弱引用就是一个指向对象的指
针，但不会增加对象本身的引用计数。可以通过 weakref 库来创建弱引用。示例如下：

In [10]:
# Class just to illustrate when deletion occurs
class Data:
    def __del__(self):
        print('Data.__del__')
# Node class involving a cycle
class Node:
    def __init__(self):
        self.data = Data()
        self.parent = None
        self.children = []
    def add_child(self, child):
        self.children.append(child)
        child.parent = self


import weakref
d= Node()
d_ref = weakref.ref(d)
print(d_ref) 

<weakref at 0x000001B09DB73040; to 'Node' at 0x000001B09CE9F6A0>


要提领（dereference）一个弱引用，可以像函数一样来调用它。如果提领后得到的对象
还依然存在，那么就返回对象，否则就返回 None。由于原始对象的引用计数并没有增
加，因此可以按照普通的方式来删除它。示例如下：


In [11]:
print(d_ref())
del d

print(d_ref())

Data.__del__
None


通过使用弱引用，就会发现因为循环引用而出现的问题都不存在了。一旦某个对象不
再被使用了，会立刻执行垃圾收集处理。